## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,2020-10-30 02:15:15,3.07,172.79,82.04,75,72,11.52,broken clouds
1,1,Dudinka,RU,2020-10-30 02:24:56,69.41,86.18,8.24,92,90,12.44,overcast clouds
2,2,Luderitz,NaN,2020-10-30 02:19:39,-26.65,15.16,56.39,74,0,27.83,clear sky
3,3,Upernavik,GL,2020-10-30 02:18:55,72.79,-56.15,22.91,97,100,5.03,light snow
4,4,Rikitea,PF,2020-10-30 02:17:50,-23.12,-134.97,75.56,73,100,6.73,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,2020-10-30 02:15:15,3.07,172.79,82.04,75,72,11.52,broken clouds
4,4,Rikitea,PF,2020-10-30 02:17:50,-23.12,-134.97,75.56,73,100,6.73,overcast clouds
5,5,Atuona,PF,2020-10-30 02:15:15,-9.80,-139.03,78.15,73,86,15.39,light rain
7,7,Vaitape,PF,2020-10-30 02:24:57,-16.52,-151.75,81.10,77,0,9.89,clear sky
11,11,Sarangani,PH,2020-10-30 02:24:58,5.40,125.46,83.75,77,57,12.86,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
541,541,Bontang,ID,2020-10-30 02:26:54,0.13,117.50,83.66,67,59,7.31,broken clouds
550,550,Sao Felix Do Xingu,BR,2020-10-30 02:25:07,-6.64,-51.99,76.03,84,68,2.44,light rain
553,553,Bolama,GW,2020-10-30 02:26:57,11.58,-15.47,82.17,69,0,6.53,clear sky
555,555,Dwarka,IN,2020-10-30 02:26:57,22.24,68.97,79.83,60,0,9.33,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                176
City                   176
Country                176
Date                   176
Lat                    176
Lng                    176
Max Temp               176
Humidity               176
Cloudiness             176
Wind Speed             176
Current Description    176
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count()

City_ID                176
City                   176
Country                176
Date                   176
Lat                    176
Lng                    176
Max Temp               176
Humidity               176
Cloudiness             176
Wind Speed             176
Current Description    176
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,82.04,broken clouds,3.07,172.79,
4,Rikitea,PF,75.56,overcast clouds,-23.12,-134.97,
5,Atuona,PF,78.15,light rain,-9.80,-139.03,
7,Vaitape,PF,81.10,clear sky,-16.52,-151.75,
11,Sarangani,PH,83.75,broken clouds,5.40,125.46,
21,Kavaratti,IN,82.87,clear sky,10.57,72.64,
22,Natal,BR,78.80,scattered clouds,-5.79,-35.21,
24,Geraldton,AU,93.20,clear sky,-28.77,114.60,
29,Srandakan,ID,86.50,clear sky,-7.94,110.25,
32,Charters Towers,AU,90.84,clear sky,-20.10,146.27,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   176
Country                176
Max Temp               176
Current Description    176
Lat                    176
Lng                    176
Hotel Name             176
dtype: int64

In [2]:
hotel_df.head()

NameError: name 'hotel_df' is not defined

In [3]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

NameError: name 'hotel_df' is not defined

In [4]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))